In [1]:
import time
import os
t = time.time()
import json
import string
import random
import math
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import nltk
# nltk.download()
import nltk, re, time
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec

In [3]:
input_dir = "../input/"
word2vec_model_dir = "./word2vec_models/"

In [4]:
train_data = pd.read_csv(input_dir+'train.csv') 
test_data = pd.read_csv(input_dir+'test.csv')

In [5]:
train_data.head(20)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
5,id22965,"A youth passed in solitude, my best years spen...",MWS
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP
7,id13515,The surcingle hung in ribands from my body.,EAP
8,id19322,I knew that you could not say to yourself 'ste...,EAP
9,id00912,I confess that neither the structure of langua...,MWS


In [6]:
print("Shape of train data:", train_data.shape)

Shape of train data: (19579, 3)


In [7]:
test_data.head(20)

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...
5,id27337,"""The thick and peculiar mist, or smoke, which ..."
6,id24265,"That which is not matter, is not at all unless..."
7,id25917,I sought for repose although I did not hope fo...
8,id04951,"Upon the fourth day of the assassination, a pa..."
9,id14549,"""The tone metaphysical is also a good one."


In [8]:
print("Shape of test data:", test_data.shape)

Shape of test data: (8392, 2)


In [9]:
# load list of stopwords
sw = set(stopwords.words("english"))
# load teh snowball stemmer
stemmer = SnowballStemmer("english")
# translator object to replace punctuations with space
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

In [10]:
print(sw)

{'has', 'but', 'no', 'over', 've', 'her', 'being', "she's", 'yours', 'am', 'does', 'can', 'some', 'd', 'other', "doesn't", 'after', 'once', 'any', 'mustn', "won't", "wasn't", 'wasn', 'its', 'ain', 'yourself', 'for', 'were', "don't", 'if', 'she', 'in', 'where', 'my', 'they', 'those', 'through', 'all', "needn't", 'needn', 'himself', 'it', 'herself', 'who', 'why', 'and', "mustn't", 'when', 'both', 'than', 'be', 'you', 'under', 'such', 'to', 'again', 'll', 'haven', 'how', 'nor', 'during', 'out', 'into', "you're", 'been', 'down', 'will', 'should', "you'd", 'between', 'until', 'doesn', 'have', 'themselves', 'doing', 'while', 'hers', "you'll", 'too', 'what', 'mightn', 'theirs', 'having', 'are', 'we', 'whom', 'hadn', "that'll", 'won', 'o', 'with', 'very', 'me', 'y', 'there', 'itself', 'just', 'yourselves', 'so', 'couldn', 'that', "weren't", 'off', 'an', 'is', 'further', 'most', 'which', "couldn't", 'his', 'your', 'about', 'did', 'hasn', 'above', 'wouldn', 'because', 'own', 's', "aren't", "shan

In [11]:
def clean_text(text):    
    """
    A function for preprocessing text
    """    
    text = str(text)    
    # replacing the punctuations with no space,which in effect deletes the punctuation marks 
    text = text.translate(translator)    
    # remove stop word
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text)    
    # stemming
    text = [stemmer.stem(word) for word in text.split()]
    text = " ".join(text)     
    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    return(text)

In [12]:
t1 = time.time()
train_data['text'] = train_data['text'].apply(clean_text)
print("Finished cleaning the train set.", "Time needed:", time.time()-t1,"sec")

Finished cleaning the train set. Time needed: 3.465871810913086 sec


In [13]:
t2 = time.time()
test_data['text'] = test_data['text'].apply(clean_text)
print("Finished cleaning the test set.", "Time needed:", time.time()-t2,"sec")

Finished cleaning the test set. Time needed: 1.551027536392212 sec


In [14]:
train_data.head(10)

,id,text,author
0,id26305,process howev afford mean ascertain dimens dun...,EAP
1,id17569,never occur fumbl might mere mistak,HPL
2,id11008,left hand gold snuff box caper hill cut manner...,EAP
3,id27763,love spring look windsor terrac sixteen fertil...,MWS
4,id12958,find noth els even gold superintend abandon at...,HPL
5,id22965,youth pass solitud best year spent gentl femin...,MWS
6,id09674,astronom perhap point took refug suggest non l...,EAP
7,id13515,surcingl hung riband bodi,EAP
8,id19322,knew could say stereotomi without brought thin...,EAP
9,id00912,confess neither structur languag code govern p...,MWS


In [15]:
test_data.head(10)

,id,text
0,id02310,still urg leav ireland inquietud impati father...
1,id24541,fire want fan could readili fan newspap govern...
2,id00134,broken frail door found two clean pick human s...
3,id27757,think possibl manag without one actual tumbl h...
4,id04081,sure limit knowledg may extend
5,id27337,thick peculiar mist smoke distinguish indian s...
6,id24265,matter unless qualiti thing
7,id25917,sought repos although hope forget knew pursu d...
8,id04951,upon fourth day assassin parti polic came unex...
9,id14549,tone metaphys also good one


In [16]:
def find_length(text):
    """
    A function to find the length
    """
    text = str(text)
    return len(text.split())

In [17]:
train_data['length'] = train_data['text'].apply(find_length)
train_data.head(10)

,id,text,author,length
0,id26305,process howev afford mean ascertain dimens dun...,EAP,21
1,id17569,never occur fumbl might mere mistak,HPL,6
2,id11008,left hand gold snuff box caper hill cut manner...,EAP,19
3,id27763,love spring look windsor terrac sixteen fertil...,MWS,21
4,id12958,find noth els even gold superintend abandon at...,HPL,16
5,id22965,youth pass solitud best year spent gentl femin...,MWS,40
6,id09674,astronom perhap point took refug suggest non l...,EAP,12
7,id13515,surcingl hung riband bodi,EAP,4
8,id19322,knew could say stereotomi without brought thin...,EAP,42
9,id00912,confess neither structur languag code govern p...,MWS,11


In [18]:
test_data['length'] = test_data['text'].apply(find_length)
test_data.head(10)

,id,text,length
0,id02310,still urg leav ireland inquietud impati father...,10
1,id24541,fire want fan could readili fan newspap govern...,28
2,id00134,broken frail door found two clean pick human s...,17
3,id27757,think possibl manag without one actual tumbl h...,20
4,id04081,sure limit knowledg may extend,5
5,id27337,thick peculiar mist smoke distinguish indian s...,17
6,id24265,matter unless qualiti thing,4
7,id25917,sought repos although hope forget knew pursu d...,12
8,id04951,upon fourth day assassin parti polic came unex...,14
9,id14549,tone metaphys also good one,5


In [19]:
train_data = pd.concat([train_data, pd.get_dummies(train_data['author'])], axis=1)
train_data.drop("author", axis = 1)
train_data.head(10)

,id,text,author,length,EAP,HPL,MWS
0,id26305,process howev afford mean ascertain dimens dun...,EAP,21,1,0,0
1,id17569,never occur fumbl might mere mistak,HPL,6,0,1,0
2,id11008,left hand gold snuff box caper hill cut manner...,EAP,19,1,0,0
3,id27763,love spring look windsor terrac sixteen fertil...,MWS,21,0,0,1
4,id12958,find noth els even gold superintend abandon at...,HPL,16,0,1,0
5,id22965,youth pass solitud best year spent gentl femin...,MWS,40,0,0,1
6,id09674,astronom perhap point took refug suggest non l...,EAP,12,1,0,0
7,id13515,surcingl hung riband bodi,EAP,4,1,0,0
8,id19322,knew could say stereotomi without brought thin...,EAP,42,1,0,0
9,id00912,confess neither structur languag code govern p...,MWS,11,0,0,1


In [20]:
train_data = train_data[~train_data['text'].isnull()]
test_data = test_data[~train_data['text'].isnull()]

/home/rahmanitrat/anaconda3/envs/deep_learning/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [21]:
train_data.to_csv(input_dir+"modified_train_data.csv",header=True, index=False)
test_data.to_csv(input_dir+"modified_test_data.csv",header=True, index=False)

In [22]:
train_comment = train_data["text"].values
test_comment = test_data["text"].values
all_comments = np.concatenate((train_comment, test_comment), axis = 0)
print("Shape of all comments:",all_comments.shape)
del(train_comment, test_comment)
all_comments = all_comments.tolist()

Shape of all comments: (27971,)


In [23]:
sentences = []
for comment in all_comments:
    comment = str(comment)
    sentences.append(comment.split())
del(all_comments)

In [24]:
t = time.time()
size = 300
window = 20
model = Word2Vec(sentences, sg = 1, size=size, window=window, workers=16, min_count=1, iter=10)
print("Finished creating word2vec model.", "Time needed:", time.time()-t,"sec")

Finished creating word2vec model. Time needed: 12.095027685165405 sec


In [25]:
fname = "word2vec_model"
model.wv.save_word2vec_format(word2vec_model_dir+fname)

In [26]:
data = {"size": size, "window":window}
with open(word2vec_model_dir+'data.json', 'w') as outfile:
    json.dump(data, outfile)